In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import logging 

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

import json
import polars as pl
from polars import col, lit
import altair as alt
import simrs
from history import History, tabular
import input_builder as b
import utils 

In [3]:
# Agents
agent_ids = ["a" +str(i) for i in range(1, 3)] 

# Goods
wheat = "Wheat"
goods = [wheat]

# Ports
genoa = "Genoa"
rome = "Rome"
milan = "Milan"
venice = "Venice"
marsailles = "Marsailles"
port_ids = [genoa, rome, milan, venice, marsailles]

# genoa -> milan -> marsailles
#   v         v
# rome -> venice
edges = [(genoa, milan), (milan, marsailles), (rome, venice), (rome, genoa), (venice, milan)]
_market = lambda net: b._market(wheat, b._market_info(net=net)) 

# net balanced
ports = [
  b._port(genoa, _market(2)), 
  b._port(milan, _market(1)), 
  b._port(rome, _market(0)), 
  b._port(venice, _market(-1)),
  b._port(marsailles, _market(-2)), 
]

_agent = lambda id, pos: b._agent(id, pos, 1000, "Greedy")
agents = [_agent(id, port_id) for (id, port_id) in zip(agent_ids, port_ids * 20)] 

x = b._inputFormat( agents = agents, ports=ports, edges= edges, opts=b._opts(ticks=100))
history = simrs.run(x)
(actions, agents, markets) = tabular(history)
markets = markets.select(pl.exclude("pricer"))

In [4]:
print("agents") 
agents.head(2)

agents


behavior,cargo,coins,id,pos,tick
str,str,f64,str,str,i64
"""Greedy""",null,1000.0,"""a1""","""Genoa""",0
"""Greedy""",null,1000.0,"""a2""","""Rome""",0


In [5]:
print("markets") 
markets.head(2)

markets


consumption,good,port,price,production,supply,tick
f64,str,str,f64,f64,f64,i64
100.0,"""Wheat""","""Venice""",100.0,99.0,1000.0,0
100.0,"""Wheat""","""Genoa""",100.0,102.0,1000.0,0


In [6]:
print("actions")
actions.head(2)

actions


action,agent_id,tick,good,port_id
str,str,i64,str,str
"""Noop""","""a1""",0,null,null
"""Noop""","""a2""",0,null,null


In [7]:
def plot_agents(agents: pl.DataFrame):
    base = alt.Chart(agents).encode(
        x='tick:Q',
        y=alt.Y('coins:Q').scale(zero=False),
        color=alt.Color('id:O').scale(scheme='dark2'),
    )
    lines = base.transform_loess('tick', 'coins', bandwidth=.5, groupby=['id']).mark_line(size=4)
    return (base.mark_point() + lines).interactive() 
  
plot_agents(agents)

alt.LayerChart(...)

In [8]:
def plot_agent_locations(agents: pl.DataFrame):
    base = alt.Chart(agents).encode(
        x='tick:Q',
        y='pos:N',
        color=alt.Color('id:O').scale(scheme='dark2'),
    )
    return base.mark_point().interactive()
plot_agent_locations(agents)

alt.Chart(...)

In [9]:
def plot_prices_by_port(ports: pl.DataFrame):
    base = alt.Chart(ports).encode(
        x='tick:Q',
        y='price:Q',
        color=alt.Color('port:O').scale(scheme='dark2'),
    )
    lines = base.transform_loess('tick', 'price', bandwidth=.5, groupby=['port']).mark_line(size=4)
    return (base.mark_point() + lines).interactive() 
plot_prices_by_port(markets)

alt.LayerChart(...)

In [18]:
## Routes 
## What are the routes that agents take?

## Pair up buy and sell actions 
actions.groupby("agent_id").agg()

In [12]:
print("actions")
actions

actions


action,agent_id,tick,good,port_id
str,str,i64,str,str
"""Noop""","""a1""",0,null,null
"""Noop""","""a2""",0,null,null
"""BuyAndMove""","""a1""",1,"""Wheat""","""Rome"""
"""BuyAndMove""","""a2""",1,"""Wheat""","""Venice"""
"""Sell""","""a1""",2,"""Wheat""",null
"""Sell""","""a2""",2,"""Wheat""",null
"""BuyAndMove""","""a1""",3,"""Wheat""","""Venice"""
"""Move""","""a2""",3,null,"""Milan"""
"""Sell""","""a1""",4,"""Wheat""",null


Metrics for individual agent
- total coins
- coins per tick
- died?

Metrics for population of agents
- min, max, median, mean, std of agent coins

How well did the agents equalize prices?
- box plot of prices
- stddev of pricesf

Construct 'trades'
- bought Cargo at StartPort for BuyPrice
- sold Cargo at EndPort for SellPrice
- profit = SellPrice - BuyPrice
- Route = StartPort -> .. ->  EndPort
- RouteLength = len(Route)